In [1]:
import sys
if '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/' not in sys.path:
    sys.path.append('/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/')
from fanc_seg_utils.annotation_interface import neuroglancer_utilities,schema_download,schema_upload,statemanager
from fanc_seg_utils.skeletons import catmaid_utilities,skeleton_manipulations,skeletonization
    
    
import os
import numpy as np
import pandas as pd
from cloudvolume import CloudVolume
import json
from annotationframeworkclient import FrameworkClient
import nglui
from concurrent import futures
from pathlib import Path

In [60]:
token = neuroglancer_utilities.get_token()
path = neuroglancer_utilities.get_cv_path(version='Dynamic_V1')
cv = CloudVolume(path['url'],use_https=True,agglomerate=False,secrets=token)

auth_token = neuroglancer_utilities.get_token(version='api')
datastack_name = 'vnc_v0' # from https://api.zetta.ai/wclee/info/

client = FrameworkClient(
    datastack_name,
    server_address = "https://api.zetta.ai/wclee",
    auth_token = auth_token
)


In [6]:
def seg_from_pt(pts,vol,image_res=np.array([4.3,4.3,45]),max_workers=4):
    ''' Get segment ID at a point. Default volume is the static segmentation layer for now. 
    Args:
        pts (list): list of 3-element np.arrays of MIP0 coordinates
        vol_url (str): cloud volume url
    Returns:
        list, segment_ID at specified point '''
    
    
    seg_mip = vol.scale['resolution']
    res = seg_mip / image_res

    pts_scaled = [pt // res for pt in pts]
    results = []
    with futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
        point_futures = [ex.submit(lambda pt,vol: vol[list(pt)][0][0][0][0], k,vol) for k in pts_scaled]
        
        for f in futures.as_completed(point_futures):
            results=[f.result() for f in point_futures]
       
    return results

In [73]:
def sv_ids_from_coords_1(coords,vol):
    sv_ids = []
    
    for coord in coords:
        sv_ids.append(vol.download_point(coord, size=1).squeeze())
    
    return np.array(sv_ids)


def sv_ids_from_coords_2(coords,vol,max_workers=5):
    sv_ids = []
    with futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
        point_futures = [ex.submit(lambda pt,vol: vol[list(pt)][0][0][0][0], k,vol) for k in pts_scaled]
        
        for f in futures.as_completed(point_futures):
            sv_ids=[f.result() for f in point_futures]
       
    return sv_ids

def sv_ids_from_coords_3(coords,vol,max_workers=5):
    sv_ids = []
    with futures.ThreadPoolExecutor(max_workers=max_workers) as ex:
        point_futures = [ex.submit(lambda pt,vol: vol.download_point(pt, size=1).squeeze(), k,vol) for k in pts_scaled]
        
        for f in futures.as_completed(point_futures):
            sv_ids=[f.result() for f in point_futures]
       
    return sv_ids

In [74]:
image_res=np.array([4.3,4.3,45])
seg_mip = cv.scale['resolution']
res = seg_mip / image_res

In [75]:
table = schema_download.download_annotation_table(client,'T1MN_somas')

In [76]:
pts_scaled = [pt // res for pt in table.pt_position.values]

In [77]:
start_1 = datetime.datetime.now()
ids_1 = sv_ids_from_coords_1(pts_scaled,cv)
end_1 = datetime.datetime.now()


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]


In [78]:
start_2 = datetime.datetime.now()
ids_2 = sv_ids_from_coords_2(pts_scaled,cv)
end_2 = datetime.datetime.now()


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]



Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.82it/s]


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]




Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]A




Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.16it/s]

Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.05it/s]



Downloading:   0%|          | 0/1 [00:00<?, ?it/s]



Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.93it/s]




Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.63it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading: 100%|██████████| 

In [79]:
start_3 = datetime.datetime.now()
ids_3 = sv_ids_from_coords_3(pts_scaled,cv)
end_3 = datetime.datetime.now()

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]



Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]



Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.80it/s]




Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]



Downloading:   0%|          | 0/1 [00:00<?, ?it/s]



Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.11it/s]




Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.93it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.03it/s]



Downloading:   0%|     

In [80]:
print(end_1-start_1)
print(end_2-start_2)
print(end_3-start_3)

0:00:53.039385
0:00:11.948986
0:00:12.413483


In [86]:
root_ids = cv.get_roots(ids_1)


{'annotationSelectionShowsSegmentation': False,
 'annotationTags': [],
 'annotations': [{'description': 'MN_A101_T1L',
   'id': '40ba86f5057c8983a550cd3804645fbf08d81fd5',
   'pointA': [13329, 114635, 1861],
   'pointB': [13429, 114735, 1871],
   'type': 'axis_aligned_bounding_box'},
  {'description': 'MN_A101_T1R',
   'id': 'a04170d3035bc2f578bbb701ffe558d36a74f6f2',
   'pointA': [63394.99609375, 114393, 2304],
   'pointB': [63494.99609375, 114493, 2314],
   'type': 'axis_aligned_bounding_box'},
  {'description': 'MN_A102_T1L',
   'id': '4443f533087db505e5d0b4d6024aac73e27a1cf2',
   'pointA': [11729, 116735, 1478],
   'pointB': [11829, 116835, 1488],
   'type': 'axis_aligned_bounding_box'},
  {'description': 'MN_A102_T1R',
   'id': 'b02dc6eb6228fa60b5b86b54b5cf0f2cc91bee97',
   'pointA': [63084.00390625, 116459, 2863],
   'pointB': [63184.00390625, 116559, 2873],
   'type': 'axis_aligned_bounding_box'},
  {'description': 'MN_A103_T1R',
   'id': '1217716e136c54ea901e2914308d68ee4144f4f

In [91]:
state = client.state.get_state_json(261002187096550146)
cell_layer = nglui.parser.get_layer(state,'MNs')

In [92]:
cell_layer

{'annotationSelectionShowsSegmentation': False,
 'annotationTags': [],
 'annotations': [{'description': 'MN_A101_T1L',
   'id': '40ba86f5057c8983a550cd3804645fbf08d81fd5',
   'pointA': [13329, 114635, 1861],
   'pointB': [13429, 114735, 1871],
   'type': 'axis_aligned_bounding_box'},
  {'description': 'MN_A101_T1R',
   'id': 'a04170d3035bc2f578bbb701ffe558d36a74f6f2',
   'pointA': [63394.99609375, 114393, 2304],
   'pointB': [63494.99609375, 114493, 2314],
   'type': 'axis_aligned_bounding_box'},
  {'description': 'MN_A102_T1L',
   'id': '4443f533087db505e5d0b4d6024aac73e27a1cf2',
   'pointA': [11729, 116735, 1478],
   'pointB': [11829, 116835, 1488],
   'type': 'axis_aligned_bounding_box'},
  {'description': 'MN_A102_T1R',
   'id': 'b02dc6eb6228fa60b5b86b54b5cf0f2cc91bee97',
   'pointA': [63084.00390625, 116459, 2863],
   'pointB': [63184.00390625, 116559, 2873],
   'type': 'axis_aligned_bounding_box'},
  {'description': 'MN_A103_T1R',
   'id': '1217716e136c54ea901e2914308d68ee4144f4f